## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
0,1,114508,8,1381006850,2013-10-05 21:00:50,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,208092,5,1586466072,2020-04-09 21:01:12,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,2,358273,9,1579057827,2020-01-15 03:10:27,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,10039344,5,1578603053,2020-01-09 20:50:53,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2,6751668,9,1578955697,2020-01-13 22:48:17,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items = user_items.query('user_id <= 20000') # the memory I could have can not affort this size of data
# To the purpose of exercising, follow results are presented only for user id less than 20,000
#user_items.head()

### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [ ]:
# Create user-by-item matrix
#user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
#user_by_movie.to_csv('./user-item-matrix-head30k.csv')

In [ ]:
#user_by_movie = pd.read_csv('./user-item-matrix-head20k.csv')
#user_by_movie.head()

In [4]:
# Read from as closed as to the original file can help to save memory too
user_by_movie = reviews[['user_id','movie_id','rating']].query('user_id <= 20000').groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [5]:
user_by_movie.head()

movie_id,91,417,439,2101,2130,2844,3863,4099,4100,4101,...,11749480,11777040,11861072,11898442,11903004,11916278,11987296,12027020,12078990,12131262
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
user_by_movie.shape, movies.shape

((20000, 20147), (35479, 35))

In [7]:
user_by_movie.loc[1].index.values

array([      91,      417,      439, ..., 12027020, 12078990, 12131262])

In [8]:
user_by_movie.loc[1][0:3]

movie_id
91    NaN
417   NaN
439   NaN
Name: 1, dtype: float64

In [9]:
tmp = user_by_movie.loc[1][0:3]
tmp[[True, True, False]].index.values

array([ 91, 417])

`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [10]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
        user_id - the user_id of an individual as int
    OUTPUT:
        movies - an array of movies the user has watched
    '''

    notnull_movies = user_by_movie.loc[user_id][user_by_movie.loc[user_id].notnull()]
    movies = notnull_movies.index.values
    
    return movies


def create_user_movie_dict():
    '''
    INPUT: 
        None
    OUTPUT: 
        movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    
    movies_seen = dict()
    for user in range(user_by_movie.shape[0]):
        movies_seen[user+1] = movies_watched(user+1)
    
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()

In [11]:
dict(list(movies_seen.items())[:5])

{1: array([114508]),
 2: array([  208092,   358273,  6751668,  7131622,  7975244,  7984734,
         8367814,  8579674,  8946378, 10039344]),
 3: array([ 790636, 1800241, 2278871, 2395417, 3344922]),
 4: array([ 267626, 1343092, 1477855, 1920849, 2024432, 2084970]),
 5: array([2884206, 3040964, 5022702])}

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [12]:
# Remove individuals who have rated 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=1):
    '''
    INPUT:  
        movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
        lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
        movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    
    movies_to_analyze = dict()
    for user, movies in movies_seen.items():
        if(len(movies) <= lower_bound):
            pass
        else:
            movies_to_analyze[user] = movies
            
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)

In [13]:
movies_to_analyze[104]

array([  117071,   292963,   964539,  1051906,  1560220,  1694021,
        3612126,  4916630,  5774060,  6169694,  6189022,  6560164,
        7343762,  8368406,  9805504, 10307440])

### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [14]:
def compute_correlation(user1, user2):
    '''
    INPUT
        user1 - int user_id
        user2 - int user_id
    OUTPUT
        the correlation between the matching ratings between the two users
    '''
    # First, we could only make the correlation calculation on movies that are both been watched by user1 and user2 
    
    # Method 1 use - set(list A.intersection(list B))
    
    movies_watched1 = set(movies_to_analyze[user1])
    movies_watched2 = set(movies_to_analyze[user2])    
    
    common_movies = list(movies_watched1.intersection(movies_watched2))
    print('Movies both user1 and user 2 watched:', common_movies)
    
    
    # Method 2 use - np.intersect1d(array A, array B, assume_unique=False), default is False, True to return index
    
    # movies_watched1 = movies_to_analyze[user1]
    # movies_watched2 = movies_to_analyze[user2]
    
    # common_movies = np.intersect1d(movies_watched1, movies_watched2, assume_unique=True)

    
    df = user_by_movie.loc[(user1, user2), common_movies]
    
    x = df.iloc[0]
    y = df.iloc[1]
    
    corr = x.corr(y)
    
    return corr #return the correlation

In [15]:
compute_correlation(2, 104)

Movies both user1 and user 2 watched: []


nan

### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [ ]:
# Which movies did both user 2 and user 104 see?


In [16]:
# What were the ratings for each user for those movies?
user_by_movie.loc[104, set(movies_to_analyze[104]).intersection(movies_to_analyze[104])]

movie_id
9805504     5.0
1051906     7.0
292963      5.0
6560164     5.0
1694021     5.0
5774060     5.0
117071      7.0
10307440    6.0
7343762     7.0
6189022     7.0
4916630     5.0
8368406     5.0
6169694     5.0
964539      6.0
1560220     7.0
3612126     5.0
Name: 104, dtype: float64

In the denominator of the correlation coefficient, we calculate the standard deviation for each user's ratings.  The ratings for user 2 are all the same rating on the movies that match with user 104.  Therefore, the standard deviation is 0.  Because a 0 is in the denominator of the correlation coefficient, we end up with a **NaN** correlation coefficient.  Therefore, a different approach is likely better for this particular situation.

`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [17]:
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
        user1 - int user_id
        user2 - int user_id
    OUTPUT
        the euclidean distance between user1 and user2
    '''
    # Same as above correlation, find out the common movies first
    
    movies_watched1 = movies_to_analyze[user1]
    movies_watched2 = movies_to_analyze[user2]  
    
    common_movies = list(movies_watched1.intersection(movies_watched2))
    
    df = user_by_movie.loc[(user1, user2), common_movies]
    
    # Use the numpy function np.linalg.norm(x_i, y_i) = sqrt( sum_ij |x_i - y_i|^2 )
    dist = np.linalg.norm(df.loc[user1], df.loc[user2])
    
    return dist #return the euclidean distance

### Using the Nearest Neighbors to Make Recommendations

You read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [18]:
# Read in solution euclidean distances"
import pickle
df_dists = pd.read_pickle("data/Term2/recommendations/lesson1/data/dists.p")

In [19]:
df_dists.head()

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


In [20]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,Western,Adventure,Drama,...,Romance,Family,Biography,Musical,Sci-Fi,Thriller,Adult,Mystery,News,Game-Show
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
templist=[8,10]
movies[movies['movie_id']==templist]['movie']

TypeError: invalid type comparison

In [22]:
movies.query('movie_id==@templist')['movie']

0    Edison Kinetoscopic Record of a Sneeze (1894)
1              La sortie des usines Lumière (1895)
Name: movie, dtype: object

In [ ]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    
    closest_neighbors = np.array(df_dists[df_dists['user1']==user].sort_values(by=['eucl_dist'])['user2'])
    
    return closest_neighbors
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
        user_id - the user_id of an individual as int
        min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
        movies_liked - an array of movies ids the user has watched and liked
    '''
  
    movies_liked = np.array(user_items.query('user_id == @user_id and rating >= @min_rating')['movie_id'])
    
    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
        movie_ids - a list of movie_ids
    OUTPUT
        movies_lst - a list of movie names associated with the movie_ids
    
    '''
    
    movies_1st = list(movies.query('movie_id==@movie_ids')['movie'])
    
    return movie_lst
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    
    user_movies = movies_watched(user)
    similar_users = find_closest_neighbors(user)
    
    recs = []
    
    for neighbor in similar_users:
        # Find out similar users' movies
        neighbor_movies = movies_liked(neighbor)
        new_recs = np.setdiff1d(neighbor_movies, user_movies, assume_unique=True)
        
        # Update recs with new_recs
        recs = np.unique(np.concatenate([recs, new_recs]))
        
        # If we have enough recommendations
        if(len(recs)>num_recs):
            break
        
    recommendations = movie_names(recs)
    
    return recommendations


def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    
    all_recs = dict()
    
    users = np.unique(df_dists['user1'])
    
    # Make the recommendations for each user
    for each_user in users:
        all_recs[each_user] = make_recommendations(each_user, num_recs)
    
    return all_recs


all_recs = all_recommendations(10)

### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [ ]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': d # letter here,
    'The two methods used to estimate user similarity were: ': e # letter here,
    'There was an issue with using the correlation coefficient.  What was it?': h # letter here
}

t.test_recs(sol_dict)

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [ ]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?': # letter here,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?': # letter here,
    'For how many users were we unable to make any recommendations for using collaborative filtering?': # letter here,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?': # letter here,
    'What might be a way for us to get 10 recommendations for every user?': # letter here   
}

t.test_recs2(sol_dict2)

In [ ]:
# Use the cells below for any work you need to do!

In [ ]:
# Users without recs
users_without_recs = []
for user, movie_recs in all_recs.items():
    if len(movie_recs) == 0:
        users_without_recs.append(user)
    
len(users_without_recs)

In [ ]:
# NaN euclidean distance values
df_dists['eucl_dist'].isnull().sum()

In [ ]:
# Users with less than 10 recs
users_with_less_than_10recs = []
for user, movie_recs in all_recs.items():
    if len(movie_recs) < 10:
        users_with_less_than_10recs.append(user)
    
len(users_with_less_than_10recs)